In [ ]:
#imports
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn import metrics

from keras_vggface.vggface import VGGFace
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#loading and performing data augmentation on the dataset
base_path = './real_fake_images_140k_subset_for_pp/'
image_gen = ImageDataGenerator(rescale=1./255.)
batch_size = 64
train_flow = image_gen.flow_from_directory(
    base_path + 'train/',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

valid_flow = image_gen.flow_from_directory(
    base_path + 'valid/',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

test_flow = image_gen.flow_from_directory(
    base_path + 'test/',
    target_size=(224, 224),
    batch_size=1,
    shuffle=False,
    class_mode='binary'
)

In [ ]:
#construct the model and add customized layers 
vgg_model = VGGFace(include_top=False, input_shape = (224,224,3))

last_layer = vgg_model.get_layer('pool5').output
flat_layer = Flatten(name='flatten')(last_layer)
fc1 = Dense(2048, activation='relu', name='fc1')(flat_layer)
dense2 = Dense(1, activation='sigmoid', name='dense2')(fc1)

vggface_model = Model(vgg_model.input, dense2)

In [ ]:
#view the model's architecture
vggface_model.summary()

In [ ]:
vggface_model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(0.0002), 
    metrics=['acc']
)

In [ ]:
#Training
train_steps = 20000//batch_size
valid_steps = 2500//batch_size

history = vggface_model.fit_generator(
    train_flow,
    epochs=5,
    steps_per_epoch=train_steps,
    validation_data=valid_flow,
    validation_steps=valid_steps
)

In [ ]:
vggface_model.save('vggface_exact_2023.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

def plot_loss(epochs, loss, val_loss):
    plt.plot(epochs, loss, 'bo', label='Training Loss')
    plt.plot(epochs, val_loss, 'orange', label = 'Validation Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()

def plot_accuracy(epochs, acc, val_acc):
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'orange', label = 'Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.show()

In [ ]:
plot_loss(range(1, len(loss) + 1), loss, val_loss)

In [ ]:
plot_accuracy(range(1, len(loss) + 1), acc, val_acc)

In [ ]:
y_pred = vggface_model.predict(test_flow)
y_test = test_flow.classes

In [ ]:
print("ROC-AUC Score:", metrics.roc_auc_score(y_test, y_pred))
print("AP Score:", metrics.average_precision_score(y_test, y_pred))
print()
print(metrics.classification_report(y_test, y_pred > 0.5))

In [ ]:
y_pred_binary = y_pred.round().astype(np.int)
con_mat_df = metrics.confusion_matrix(y_test, y_pred_binary)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix=con_mat_df, display_labels=['0:fake', '1:real'])
cm_display.plot()
plt.show()

In [ ]:
x, y = test_flow.next()
print(f"Predicted likelihood: {vggface_model.predict(x)[0][0]:.4f}")
print(f"Actual label: {int(y[0])}")
print(f"\nCorrect Prediction: {round(vggface_model.predict(x)[0][0])==y[0]}")
plt.imshow(np.squeeze(x))

In [ ]:
metrics.accuracy_score(y_test, y_pred_binary)

In [ ]:
# Evaluate the model on the test data
print("Evaluate on test data")
results = vggface_model.evaluate(test_flow, batch_size=64)
print("test loss, test acc:", results)